In [1]:
import random
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
import numpy as np
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score
from itertools import groupby
import copy
import time
from tqdm.notebook import tqdm

In [2]:
#importing the data and normalizing the x variable to between 0 and 1
iris = pd.DataFrame(pd.read_csv("iris.csv"))
x_data = iris.iloc[:,0:4]
y_data = iris.iloc[:,4:5]
min_max_scaler = preprocessing.MinMaxScaler()
x_data = pd.DataFrame(min_max_scaler.fit_transform(x_data))

#splitting into test and train sets
x_train_orig, x_test, y_train_orig, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_orig, y_train_orig, test_size=0.25, random_state=42)

# Functions to generate random starting trees

## Function to define random starting tree or assign points to a given tree

In [3]:
def random_tree(x, y, method, given_splits = "missing", given_cutoffs = "missing"):

    #calculate a randomized tree if method is 0, otherwise classify points if tree given
    if method == 0:
        #holding the tree structure
        splits = []
        cutoffs = []
        
        #iterate to generate all of the split nodes randomly
        p = len(x_train.columns)
        depth = random.randint(math.floor(p/2),p)
        tot_split = 2 ** (depth) - 1

        #creating list of parent nodes to use to check if a parent node has an early leaf
        parent_node = [0]
        for i in range(2 ** depth - 1):
            parent_node += [i, i]
            
        for split in range(tot_split):
            #checking if the parent node of this split is already an early leaf
            if split != 0 and splits[parent_node[split]] == ["early_leaf", "early_leaf"]:
                splits.append(["early_leaf", "early_leaf"])
                cutoffs.append("early_leaf")
            else:
                #potentially adding early leaf randomly
                if random.uniform(0,1) < 0.1:
                    splits.append(["early_leaf", "early_leaf"])
                    cutoffs.append("early_leaf")
                #otherwise create real split instead of early leaf
                else:
                    #save random split var and cutoff to list
                    idx = random.randint(0,p-1)
                    min_value = x.iloc[:,idx].min()
                    max_value = x.iloc[:,idx].max()
                    cut_off = random.uniform(max_value, min_value)

                    #making formation easy to alter for later hyperplane usage
                    splits.append([idx, 1])
                    cutoffs.append(cut_off)
    else:
        splits = given_splits
        cutoffs = given_cutoffs
        tot_split = len(splits)
        depth = int(math.log2(tot_split + 1))
        #creating list of parent nodes to use to check if a parent node has an early leaf
        parent_node = [0]
        for i in range(2 ** depth - 1):
            parent_node += [i, i]
        
    #calculating the leaf nodes points are assigned to
    assignments = []
    pts = []
    leaf_tracks = []
    for i in range(x.shape[0]):
        #point number, passing in point, starting node, split info, cutoff info, minimum number node for index of a leaf
        leaf_num = assign_leaf(x.iloc[i,:], 0, splits, cutoffs, tot_split)
        pts.append(i)
        leaf_tracks.append(leaf_num)
    assignments.append(pts)
    assignments.append(leaf_tracks)
        
    #finding the classification of each leaf by majority
    decisions = leaf_classification(assignments, y)
    
    #with the assignment of each point and the decision of each leaf, predict each points assignment
    decision_by_point = []
    for i in range(len(assignments[0])):
        decision_by_point.append(decisions[assignments[1][i]])
    assignments.append(decision_by_point)
    
    #returning the results
    return splits, cutoffs, assignments, decisions, parent_node, depth, tot_split
        

## Assigns points to a given tree, called by random_tree

In [4]:
def assign_leaf(point, splt_indx, splits, cutoffs, leaf_indx):
    #reading in the list of parameters and their coefficients to evaluate versus the cutoff
    splt_list = splits[splt_indx]
    cur_cutoff = cutoffs[splt_indx]
    
    #calculating the current value for the given point
    tot_val = 0
    #multiplying the value of the point by its coefficient in the split list to check for which direction it goes
    for i in range(int(len(splt_list) / 2)):
        col_num = splt_list[2*i]
        coeff = splt_list[2*i+1]
        #dont bother if its an early leaf
        if col_num == "early_leaf":
            tot_val = "early_leaf"
            break
        else:
            tot_val += point[col_num] * coeff
    if tot_val == "early_leaf" or cur_cutoff == "early_leaf":
        return splt_indx
    #comparing value to current cutoff and assigning to upper or lower leaf, finding next node to visit
    elif tot_val >= cur_cutoff:
        next_node = 2 * splt_indx + 2
    else:
        next_node = 2 * splt_indx + 1
        
    #if next node is leaf, terminate, otherwise recursion until find leaf node
    if next_node >= leaf_indx:
        #returning assigned leaf number
        return next_node
    else:
        return assign_leaf(point, next_node, splits, cutoffs, leaf_indx)


## Decides leaf decisions, called by random_tree

In [5]:
def leaf_classification(assign, y):
    #splitting into sets of the points and their assignments
    assignments = assign[1]
    pts = assign[0]
    
    #finding the unique leaves represented
    cur_leaves = list(set(assignments))
    
    #finding the count of each group per leaf and assigning the leaf its majority classification
    classification = []
    #for each leaf we are checking
    for leaf in cur_leaves:
        #for all the points in our dataset
        indices = []
        for idx in range(len(pts)):
            #keep track of which points are in this leaf
            if assignments[idx] == leaf:
                indices.append(idx)
        #we know all points in leaf, find majority decision and assign it
        pts_left = y.iloc[indices]
        decision = pts_left.iloc[:,0].max()
        classification.append(leaf)
        classification.append(decision)
    
    #return the results as a dictionary
    return {classification[i]: classification[i + 1] for i in range(0, len(classification), 2)}


## Miscellaneous functions to help with others

In [6]:
#function to find the baseline misclassified error
def baseline_error(y_true, y_pred):
    tot = 0
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i]:
            tot += 1
    return tot

#optional beta, if included the loss function will have the 0 norm penalizer
def loss_function(y_true, y_pred, base_err, tot_splits, splits, alpha, beta = 0):
    #finding the total number of misclassified points in the new tree
    tot_points = len(y_true)
    L = 0
    for point in range(tot_points):
        if y_true[point] != y_pred[point]:
            L += 1
    
    #finding the amount of variables used in each split for total number used
    tot_vars_used = 0
    for split in splits:
        if split[0] == "early_leaf":
            continue
        else:
            vars_used = np.count_nonzero(split[1::2])
            tot_vars_used += vars_used
    
    #returning the loss function
    return (L / base_err) + alpha * tot_splits + beta * tot_vars_used

#function to return list of all parents of a leaf node for the RR function to use
def children(orig_node, cur_node, parent_nodes, all_children):
    #append new node the list of children
    all_children.append(int(cur_node))
    #Finding the next nodes to the left and right
    cur_node_right = 2 * cur_node + 2
    cur_node_left = 2 * cur_node + 1
    
    #checking if they are in the tree, if not add them
    if int(cur_node_right) not in set(all_children) and cur_node_right <= len(parent_nodes) - 1:
        return children(orig_node, cur_node_right, parent_nodes, all_children)
    if int(cur_node_left) not in set(all_children) and cur_node_left <= len(parent_nodes) - 1:
        return children(orig_node, cur_node_left, parent_nodes, all_children)
    
    #if both of the splits are in the tree, go up another level and try again
    if cur_node % 2 == 0:
        cur_node_up = (cur_node - 2) / 2
    else:
        cur_node_up = (cur_node - 1) / 2
    #as long as we have more branches to check and are not above our original node, check them
    if cur_node_up >= orig_node:
        return children(orig_node, cur_node_up, parent_nodes, all_children)
    
    #return final list of children
    return set(all_children)                                                 
                                                                

## Find the left or right subtree from a given node for random restarts

In [7]:
#returns the left or right subtree of a node
def delete_node(node_num, splits, cutoffs, parent_node, decisions, split_direction):
    #branch node
    if len(splits) > 1:
        #left split
        if split_direction == 0:
            sub_tree_start = 2 * node_num + 1
        #right split
        else:
            sub_tree_start = 2 * node_num + 2

        #getting subtree indexes
        subtree_indx = list(children(sub_tree_start, sub_tree_start, parent_node, []))
        
        new_splits = []
        new_cutoffs = []
        for i, node in enumerate(subtree_indx):
            if node < len(splits):
                new_splits.append(splits[node])
                new_cutoffs.append(cutoffs[node])
        return new_splits, new_cutoffs, decisions
        
    #if it is a final split node change to an early leaf
    else:
        splits[0] = ["early_leaf", "early_leaf"]
        cutoffs[0] = "early_leaf"
        
        return splits, cutoffs, decisions
    

## Functions to find optimal splits within the random_restart function

### General function to update the tree after split changes are made

In [8]:
def update_tree(x, y, first_node, parent_nodes, splits, cutoffs, new_splits, new_cutoffs, took):
        #if parallel change, update directly
        if took == 1 or took == 4:
            splits[first_node] = new_splits[0]
            cutoffs[first_node] = new_cutoffs[0]
        #if left or right subtree, update here
        else:
            #finding parent node of starting split
            start_node = parent_nodes[first_node]
            all_children_nodes = list(children(start_node, start_node, parent_nodes, []))
            all_children_nodes = all_children_nodes[0:int(len(all_children_nodes) / 2)]
            #only keeping the split nodes (excluding children)
            sub_children = list(children(first_node, first_node, parent_nodes, []))
            sub_children = sub_children[0:int(len(sub_children) / 2)]
            #change nodes in original to corresponding ne
            i = 0
            for node in (all_children_nodes):
                if node in sub_children:
                    if i < len(new_splits):
                        splits[node] = new_splits[i]
                        cutoffs[node] = new_cutoffs[i]
                        i +=1
                    else:
                        splits[node] = ["early_leaf", "early_leaf"]
                        cutoffs[node] = "early_leaf"
                    
        _, _, assign, decisions, _, _, _ = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
        return splits, cutoffs

## Finding the optimal parallel splits amongst the critical cutoff values

In [9]:
#pass in the subtree to optimize, always will change the first splits / cutoffs
def best_parallel_split(x, y, splits, cutoffs, decisions, minleafsize, base_error, alpha, beta):
    n, p = x.shape
    error_best = math.inf
    best_splits = copy.deepcopy(splits)
    best_cutoffs = copy.deepcopy(cutoffs)
    best_decisions = copy.deepcopy(decisions)
    
    #loop over all dimensions
    for j in range(p):
        x_vals = x.iloc[:,j]
        x_vals = sorted(list(x_vals))
        
        #loop over all possible split placements (between x values)
        for i in range(n - 1):
            new_split_cutoff = 0.5 * (x_vals[i] + x_vals[i+1])
            
            #updating for new cutoff and split variable
            cutoffs[0] = new_split_cutoff
            splits[0] = [j, 1]
            
            #finding where new point assignments are
            _, _, assigns, decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
            min_leaf_num = min([len(list(group)) for key, group in groupby(sorted(assigns[1]))])
            
            #if min leaf size constraint held, make the switch
            if min_leaf_num >= minleafsize:
                cur_error = loss_function(list(y.iloc[:,0]), assigns[2], base_error, tot_splits, splits, alpha, beta)
                if cur_error < error_best:
                    best_splits = copy.deepcopy(splits)
                    best_cutoffs = copy.deepcopy(cutoffs)
                    best_decisions = copy.deepcopy(decisions)
                    error_best = cur_error
                    
                    
    #returning the best split
    return best_splits, best_cutoffs, best_decisions

## finding the best hyperplane splits amongst the ciritcal values U and deletion values W

In [10]:
def besthyperplanesplit(x, y, splits, cutoffs, minleafsize, base_error, alpha, beta):
    #getting dimensions of the data for later use
    n, p = x.shape
    best_splits = copy.deepcopy(splits)
    best_cutoffs = copy.deepcopy(cutoffs)
    
    
    #editting the split format now to make for easier manipulation later
    #checking if current split is an early leaf to initialize changing_split correctly
    if splits[0] == ["early_leaf", "early_leaf"]:
        changing_split = []
        for dimension in range(p):
            changing_split += [dimension, 0]
    else:    
        vars_used_split = splits[0][::2]
        coefs_used_split = splits[0][1::2]
        changing_split = []
        for dimension in range(p):
            if dimension not in set(vars_used_split):
                changing_split += [dimension, 0]
            else:
                indx = vars_used_split.index(dimension)
                changing_split += [dimension, coefs_used_split[indx]]
    best_changing_split = copy.deepcopy(changing_split)
    
    #finding initial input error
    _, _, assigns, decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
    error_best = loss_function(list(y.iloc[:,0]), assigns[2], base_error, tot_splits, splits, alpha, beta)
    prev_error = error_best + 1
    
    #starting the while loop to find the best hyperplane split
    while error_best != prev_error:
        prev_error = error_best #not sure about this line
        
        #checking for a new hyperplane split against all dimensions
        for i in range(p):
            
            #calculating critical values for perturbating
            U = []
            for j in range(n):
                #getting the jth point of the ith dimension to calculate the corresponding V and U's
                if x.iloc[j, i] != 0:
                    if cutoffs[0] == "early_leaf":
                        cutoffs[0] = 0
                        V = sum(changing_split[2*k+1] * x.iloc[j,k] - cutoffs[0] for k in range(int(len(changing_split)/2)))
                    else:
                        V = sum(changing_split[2*k+1] * x.iloc[j,k] - cutoffs[0] for k in range(int(len(changing_split)/2)))
                    U.append((changing_split[2*i+1] * x.iloc[j, i] - V) / x.iloc[j, i])
            U = sorted(U)
            
            #looping over all split placements
            for j in range(len(U) - 1):
                c = 0.5 * (U[j] + U[j+1])
                changing_split[2*i+1] = c
                splits[0] = changing_split
                
                #finding where new point assignments are
                _, _, assigns, decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
                min_leaf_num = min([len(list(group)) for key, group in groupby(sorted(assigns[1]))])
                
                #if min leaf size constraint held, make the switch
                if min_leaf_num >= minleafsize:
                    cur_error = loss_function(list(y.iloc[:,0]), assigns[2], base_error, tot_splits, splits, alpha, beta)
                    if cur_error < error_best:
                        best_changing_split = copy.deepcopy(changing_split)
                        best_splits = copy.deepcopy(splits)
                        best_cutoffs = copy.deepcopy(cutoffs)
                        error_best = cur_error
            
            #updating splits to test reduction of variables on to only be the best found through split placements procedure
            changing_split = copy.deepcopy(best_changing_split)
            
            #checking if a split should be deleted to improve error
            #check if dimension is still in the split
            if changing_split[2*i+1] != 0:
                #calculating the critical values for deletion
                W = []
                #finding critical deletion values for all points
                for j in range(n):
                    V = sum(changing_split[2*k+1] * x.iloc[j,k] - cutoffs[0] for k in range(int(len(changing_split)/2)))
                    W.append(V + cutoffs[0] - changing_split[2*i+1])
                W = sorted(W)
                
                #finding best W critical value to use for b
                changing_split[2*i+1] = 0
                for j in range(len(W) - 1):
                    b = 0.5*(W[j] + W[j+1]) 
                    cutoffs[0] = b
                    splits[0] = changing_split
                    
                    #finding where new point assignments are
                    _, _, assigns, decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
                    min_leaf_num = min([len(list(group)) for key, group in groupby(sorted(assigns[1]))])

                    #if min leaf size constraint held, make the switch
                    if min_leaf_num >= minleafsize:
                        cur_error = loss_function(list(y.iloc[:,0]), assigns[2], base_error, tot_splits, splits, alpha, beta)
                        if cur_error < error_best:
                            best_changing_split = copy.deepcopy(changing_split)
                            best_splits = copy.deepcopy(splits)
                            best_cutoffs = copy.deepcopy(cutoffs)
                            error_best = cur_error
            
            #updating splits and cutoffs used for next iteration only to be best found through deletion procedure
            changing_split = copy.deepcopy(best_changing_split)
            cutoffs = copy.deepcopy(best_cutoffs)
                    
    #returning the results
    return best_splits, best_cutoffs
                
                

## Finding the best option between parallel, hyperplane, left, or right subtree splits

In [11]:
def optimizenodehyper(x, y, splits, cutoffs, decisions, base_error, alpha, beta, minleafsize):
    #finding the initial error of input tree
    _, _, assign, _, parent_node, _, tot_splits = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
    error_best = loss_function(list(y.iloc[:,0]), assign[2], base_error, tot_splits, splits, alpha, beta)
    final_splits, final_cutoffs, final_decisions, took = splits, cutoffs, decisions, 0
    #getting the best parallel split option
    parallel_splits, parallel_cutoffs, parallel_decisions = best_parallel_split(x, y, splits, cutoffs, decisions, minleafsize, base_error, alpha, beta)
    _, _, assign_parallel, parallel_decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = parallel_splits, given_cutoffs = parallel_cutoffs)
    parallel_error = loss_function(list(y.iloc[:,0]), assign_parallel[2], base_error, tot_splits, parallel_splits, alpha, beta)
    if parallel_error < error_best:
        final_splits, final_cutoffs, final_decisions, took = parallel_splits, parallel_cutoffs, parallel_decisions, 1
        error_best = parallel_error
    
    #making sure it is feasbile to cut the split out
    #getting the left subtree errors
    left_subtree, left_cutoffs, left_decisions = delete_node(0, splits, cutoffs, parent_node, decisions, 0)
    _, _, assign_left, left_decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = left_subtree, given_cutoffs = left_cutoffs)
    left_error = loss_function(list(y.iloc[:,0]), assign_left[2], base_error, tot_splits, left_subtree, alpha, beta)
    #saving left subtree if it is better than current loss
    if left_error < error_best:
        final_splits, final_cutoffs, final_decisions, took = left_subtree, left_cutoffs, left_decisions, 2
        error_best = left_error

    #getting right subtree errors
    right_subtree, right_cutoffs, right_decisions = delete_node(0, splits, cutoffs, parent_node, decisions, 1)
    _, _, assign_right, right_decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = right_subtree, given_cutoffs = right_cutoffs)
    right_error = loss_function(list(y.iloc[:,0]), assign_right[2], base_error, tot_splits, right_subtree, alpha, beta)
    #saving right subtree if it is better than current
    if right_error < error_best:
        final_splits, final_cutoffs, final_decisions, took = right_subtree, right_cutoffs, right_decisions, 3
        error_best = right_error
    
    #beginning hyperplane restarts loop
    H = 5
    for h in range(H):
        if h == 0:
            split_start = copy.deepcopy(splits)
            cutoffs_start = copy.deepcopy(cutoffs)
        else:
            #getting a random hyperplane start
            split_start = copy.deepcopy(splits)
            random_split = []
            for i in range(len(x.columns)):
                random_split.append(i)
                random_split.append(random.uniform(min(x.iloc[:,i]), max(x.iloc[:,i])))
            split_start[0] = random_split
            
            #getting a random cutoff start
            cutoffs_start = copy.deepcopy(cutoffs)
            max_num = sum(max(x.iloc[:, i]) for i in range(len(x.columns)))
            cutoffs_start[0] = random.uniform(-max_num, max_num)
        
        #getting the best hyperplane split
        hyper_splits, hyper_cutoffs = besthyperplanesplit(x, y, split_start, cutoffs_start, minleafsize, base_error, alpha, beta)
        _, _, assign_hyper, hyper_decisions, _, _, tot_splits = random_tree(x, y, 1, given_splits = hyper_splits, given_cutoffs = hyper_cutoffs)
        hyper_error = loss_function(list(y.iloc[:,0]), assign_hyper[2], base_error, tot_splits, hyper_splits, alpha, beta)
        if hyper_error < error_best:
            final_splits, final_cutoffs, final_decisions, took = copy.deepcopy(hyper_splits), copy.deepcopy(hyper_cutoffs), copy.deepcopy(hyper_decisions), 4
            error_best = copy.deepcopy(hyper_error)

    #outputting the best tree found
    return final_splits, final_cutoffs, final_decisions, error_best, took

# Random Restart Master function

In [12]:
def random_restarts(x, y, num_restarts, alpha, beta, minleafsize):
    #creating lists to save the results of the random restart algorithm
    saved_errors = []
    saved_splits = []
    saved_cutoffs = []
    saved_decisions = []
    
    #iterating through each restart for algorithm
    for restart_iteration in tqdm(range(num_restarts), desc="Training Tree with Alpha: {} and Beta: {}".format(alpha, beta)):
        #generating a random tree to start the algorithm and finding current error
        splits, cutoffs, assign, decisions, parent_node, depth, tot_splits = random_tree(x, y, 0)
        #generate baseline for loss function calculation
        base_err = baseline_error(list(y.iloc[:,0]), assign[2])
        #finding loss function value of original random tree
        cur_error = loss_function(list(y.iloc[:,0]), assign[2], base_err, tot_splits, splits, alpha, beta)
        error_prev = -1
        
        #iterate until the current and previous errors are the same (no change found in the tree)
        while cur_error != error_prev:
            #finding loss function on current tree given to function
            error_prev = cur_error
            
            #randomly iterating through all split nodes
            nodes = list(range(tot_splits))
            random.shuffle(nodes)
            all_errors = []
            for node in nodes:
                orig_splits = copy.deepcopy(splits)
                orig_cutoffs = copy.deepcopy(cutoffs)
                
                #finding points currently assigned to this node by the tree
                point_indx = []
                for point in range(len(y)):
                    #if leaf in current node, keep point
                    if assign[1][point] in children(node,node,parent_node, []):
                        point_indx.append(point)
                #creating set of x and y points left for this node iteration
                cur_x = x.iloc[point_indx, :]
                cur_y = y.iloc[point_indx]
                
                #if no points assigned to node, changing node wont help error
                if len(cur_x) > 0:
                    #getting subtree structure for current node
                    subtree_indx = list(children(node, node, parent_node, []))
                    new_splits = []
                    new_cutoffs = []
                    for i, t in enumerate(subtree_indx):
                        if t < len(splits):
                            new_splits.append(splits[t])
                            new_cutoffs.append(cutoffs[t])
                    

                    #printing local subtree error
                    _, _, cur_assign, _, _, _, tot_splits = random_tree(cur_x, cur_y, 1, given_splits = new_splits, given_cutoffs = new_cutoffs)
                    subtree_error = loss_function(list(cur_y.iloc[:,0]), cur_assign[2], base_err, tot_splits, new_splits, alpha, beta) 

                    #now insert function to find best replacement for current node
                    final_splits, final_cutoffs, final_decisions, error_best, took = optimizenodehyper(cur_x, cur_y, new_splits, new_cutoffs, decisions, base_err, alpha, beta, minleafsize)
                    
                    #update original tree with the best new split found in the optimizenodehyper function
                    if took != 0:
                        splits, cutoffs = update_tree(cur_x, cur_y, node, parent_node, splits, cutoffs, final_splits, final_cutoffs, took)
                    _, _, cur_assign, _, _, _, tot_splits = random_tree(cur_x, cur_y, 1, given_splits = splits, given_cutoffs = cutoffs)
                    cur_error = loss_function(list(cur_y.iloc[:,0]), cur_assign[2], base_err, tot_splits, splits, alpha, beta)
                
                #update error of entirely update tree
                _, _, cur_assign, _, _, _, tot_splits = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
                cur_error = loss_function(list(y.iloc[:,0]), cur_assign[2], base_err, tot_splits, splits, alpha, beta)
                #print("\t\tCurrent Loss Middle:\t", cur_error)
                if not len(all_errors) == 0 and cur_error > all_errors[-1]:
                    splits = copy.deepcopy(orig_splits)
                    cutoffs = copy.deepcopy(orig_cutoffs)
                else:
                    all_errors.append(cur_error)

            #update error of entirely update tree
            _, _, cur_assign, _, _, _, tot_splits = random_tree(x, y, 1, given_splits = splits, given_cutoffs = cutoffs)
            cur_error = loss_function(list(y.iloc[:,0]), cur_assign[2], base_err, tot_splits, splits, alpha, beta)
            
            #updating the vectors for next iteration
            assign = cur_assign
            
        #once the errors converage and no changes are made to the tree, return our locally optimal tree
        #append tree to a list
        saved_splits.append(copy.deepcopy(splits))
        saved_cutoffs.append(copy.deepcopy(cutoffs))
        saved_decisions.append(copy.deepcopy(decisions))
        
        #append its loss function value to a list
        saved_errors.append(cur_error)
        
    #return the best of all of our locally optimal trees here
    #check for the index of the minimum loss function
    output_error = min(saved_errors)
    indx = saved_errors.index(output_error)
    
    #return the tree corresponding with that index
    return saved_splits[indx], saved_cutoffs[indx], saved_decisions[indx], output_error

In [ ]:
#running the random restart algorithm
time_taken = []
validation_accuracy = []
parameter_combo = []

alphas = [0, 0.1, 0.01, 0.001]
betas = [0, 0.33333]
for alpha in tqdm(alphas, desc = "Alpha Progress"):
    for beta in tqdm(betas, desc = "Beta Progress with Alpha: {}".format(alpha)):
        start = time.time()
        splits, cutoffs, decisions, loss_value = random_restarts(x_train, y_train,5,alpha,beta, 1) 
        end = time.time()
        
        #saving time it took
        time_taken.append(start - end)
        
        #saving alpha value and beta value for the run
        parameter_combo.append([alpha, beta])
        
        #saving validation accuracy
        splits, cutoffs, assign, decisions, parent_node, depth, tot_splits = random_tree(x_valid, y_valid, 1, given_splits = splits, given_cutoffs = cutoffs)
        validation_accuracy.append(accuracy_score(y_valid, assign[2]))
        

In [ ]:
#getting final accuracy stats
splits, cutoffs, assign, decisions, parent_node, depth, tot_splits = random_tree(x_train_orig, y_train_orig, 1, given_splits = splits, given_cutoffs = cutoffs)
in_sample = accuracy_score(y_train_orig, assign[2])
splits, cutoffs, assign, decisions, parent_node, depth, tot_splits = random_tree(x_test_orig, y_test_orig, 1, given_splits = splits, given_cutoffs = cutoffs)
out_sample = accuracy_score(y_test, assign[2])

print("In Sample:\t", in_sample)
print("Out Sample:\t", out_sample)